In [1]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from lsst.daf.butlerUtils import ExposureIdInfo

import lsst.afw.display            as afwDisplay
import lsst.afw.image              as afwImage
import lsst.daf.base               as dafBase
import lsst.afw.geom               as afwGeom
import lsst.afw.table              as afwTable

import lsst.meas.algorithms        as measAlg

import lsst.afw.display.utils as afwDisplayUtils
import lsst.afw.image.utils   as afwImageUtils


from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.pipe.tasks.calibrate         import CalibrateTask
from lsst.meas.algorithms.detection    import SourceDetectionTask
from lsst.meas.deblender               import SourceDeblendTask
from lsst.meas.base                    import SingleFrameMeasurementTask

In [3]:
disp  = afwDisplay.Display(1)

In [4]:
schema = afwTable.SourceTable.makeMinimalSchema()
algMetadata = dafBase.PropertyList()

config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask =         CharacterizeImageTask(None, config=config)

config = SourceDetectionTask.ConfigClass()
if True:
    config.thresholdValue = 100       # detection threshold in units of thresholdType
    if True:
        print "SourceDetectionTask.thresholdType: %s" % (
            SourceDetectionTask.ConfigClass.thresholdType.__doc__)
    config.thresholdType = "stdev"   # units for thresholdValue
if False:                    
    config.doTempLocalBackground = True  # Use local-background during detection step
sourceDetectionTask =   SourceDetectionTask(schema=schema, config=config)

sourceDeblendTask =     SourceDeblendTask(schema=schema)

config = SingleFrameMeasurementTask.ConfigClass()
config.doApplyApCorr = 'yes'
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema, config=config,
                                                   algMetadata=algMetadata)

SourceDetectionTask.thresholdType: specifies the desired flavor of Threshold
Allowed values:
	pixel_stdev	threshold applied to per-pixel std deviation
	variance	threshold applied to image variance
	value	threshold applied to image value
	stdev	threshold applied to image std deviation



In [5]:
fileName = "example2.fits"
exposureIdInfo = ExposureIdInfo(0, 5)

exposure = afwImage.ExposureF(fileName)

tab = afwTable.SourceTable.make(schema)

result = charImageTask.characterize(exposure, exposureIdInfo)

result = sourceDetectionTask.run(tab, exposure)
sources = result.sources

sourceDeblendTask.run(exposure, sources, exposure.getPsf())

sourceMeasurementTask.run(exposure, sources)

sources.writeFits("outputTable.fits")
exposure.writeFits("example2-out.fits")

In [6]:
sources = sources.copy(True)

good = np.logical_and.reduce([sources.get('base_PixelFlags_flag_saturatedCenter') == 0,
                              sources.get("deblend_nChild") == 0,
                              ])

In [7]:
if True:
    disp.mtv(exposure)
else:
    disp.erase()
    
disp.pan(1163, 533); disp.zoom(1)

with disp.Buffering():
    for s in sources[good]:
        disp.dot('+', *s.getCentroid(), ctype=afwDisplay.RED)